In [ ]:
import pandas as pd
import shutil
from pandera.errors import SchemaErrors

from src.preprocesamiento.preprocesamiento import validar_y_limpieza
from src.preprocesamiento.pandera import build_schema
from src.config.config import load_catalog


In [ ]:
widget = 'train or predict'

In [ ]:
catalog, params = load_catalog(env="base")
schema = build_schema(params['data_validation'])

validated_df = schema.validate(catalog.load('titanic_intermediate'),lazy=True)

print("Schema validado correctamente")

Schema validado correctamente


In [ ]:
catalog, params = load_catalog(env="base")
if widget == 'train':

    shutil.move("data/titanic_train.csv", "data/raw/titanic_train.csv")
    # Validar que el nuevo dataset cumpla con el schema definido
    df_train = catalog.load('titanic_raw')
    try:
        schema = build_schema(params['data_validation'])
        validated_df = schema.validate(df_train,lazy=True)
    except SchemaErrors as e:
        print("Errores encontrados en el schema")
        print(e.failure_cases)
        exit(1)
    # Preprocesar el dataset
    df_limpio = validar_y_limpieza(df_train)
    catalog.save(data=df_limpio, ds_name='titanic_intermediate')


elif widget == 'predict':

    shutil.move("data/titanic_test.csv", "data/raw/titanic_test.csv")
    # Validar que el nuevo dataset cumpla con el schema definido
    try:
        df_predict = catalog.load('to_predict_raw')
        validated_df = schema.validate(df_predict,lazy=True)
    except SchemaErrors as e:
        print("Errores encontrados en el schema")
        print(e.failure_cases)
        exit(1)
    # Preprocesar el dataset
    df_limpio = validar_y_limpieza(df_predict)
    catalog.save(data=df_limpio, ds_name='to_predict_intermediate')
